# Mandatory Functions

In [21]:
#Fuzzy comparer
#mandatory! Run first!
ambig={'a':['a','','',''],
          'c':['','c','',''],
          'g':['','','g',''],
          't':['','','','t'],
          'y':['','c','','t'],
          'r':['a','','g',''],
          'w':['a','','','t'],
          's':['','c','g',''],
          'k':['','','g','t'],
          'm':['a','c','',''],
          'd':['a','','g','t'],
          'v':['a','c','g',''],
          'h':['a','c','','t'],
          'b':['','c','g','t'],
          'n':['a','c','g','t']}
rev={'a':'a',
        'c':'c',
        'g':'g',
        't':'t',
        'ct':'y',
        'ag':'r',
        'at':'w',
        'cg':'s',
        'gt':'k',
        'ac':'m',
        'agt':'d',
        'acg':'v',
        'act':'h',
        'cgt':'b',
        'acgt':'n'}
def fuzz(ref,sub,threshold):
    fscore=0
    cons=""
    #if the reference starts with cc it is a site and needs to be checked
    if sites[sub][4]=='R':
        for i in range(len(ref)):
            #no ambiguity for the PAM site
            if i<2:
                cons=cons+'c'
                continue
            match=""
            #basically check the presence of a binary value of whether the reference and subject bases have other bases (if ambiguous)
            for j in range(4):
                if ambig[ref[i]][j]==ambig[sub[i]][j]:
                    match+=(ambig[ref[i]][j])
                else:    
                    match+=(ambig[ref[i]][j]+ambig[sub[i]][j])
            #creates a "match" which is a list of all the bases present at a locus
            #interprets the "match" from the rev list
            cons=cons+rev[match]
            #this checks if the base is ambiguous and adds 1 to the ambiguity counter if it is
            if match!=rev[match]:
                if i==2:
                    continue
                fscore+=1
            #if too many ambiguities, just stops
            if fscore > threshold:
                break
    #same as cc but for gg at the end
    elif sites[sub][4]=='F':
        for i in range(len(ref)):
            if i>20:
                cons=cons+'g'
                continue
            match=""
            for j in range(4):
                if ambig[ref[i]][j]==ambig[sub[i]][j]:
                    match+=(ambig[ref[i]][j])
                else:    
                    match+=(ambig[ref[i]][j]+ambig[sub[i]][j])
            cons=cons+rev[match]
            if match!=rev[match]:
                if i==20:
                    continue
                fscore+=1
            if fscore > threshold:
                break
    #if too many ambiguites returns false
    if fscore > threshold:
        return 'False'
    #otherwise return the consensus
    else:
        return cons  

In [15]:
#selector
def select(buffer,newin,endwin):
    clear=False
    subuffer=[]
    sel=[]
    #if the buffer is longer than the desited distance
    if float(endwin)-newin>dist:
        tar=(float(endwin)-newin)/(int((float(endwin)-newin)/dist))
        #create evenly divided subsets
        for d in range(int((float(endwin)-newin)/dist)):
            for c in buffer:
                if newin+tar*d <temp[temp.index(c)-1]<newin+tar*(d+1):
                    subuffer.append(c)
            #select normally from subset
            while clear==False:
                clear=True
                for b in range(len(subuffer)):      
                    try:
                        #if the next sequence in the buffer has a higher number of matches, remove current sequence
                        if int(sites[subuffer[b]][0])<int(sites[subuffer[b+1]][0]):
                            clear=False
                            subuffer.pop(b)
                            break
                        #if the previous sequence in the buffer has a higher number of matches, remove current sequence
                        elif int(sites[subuffer[b]][0])<int(sites[subuffer[b-1]][0]):
                            clear=False
                            subuffer.pop(b)
                            break
                    except:
                        pass
            #select the middle one of the remainder
            if len(subuffer)>0:
                sel.append(subuffer[int(len(subuffer)/2)])
    #if it isn't over dist select normally
    else:
        while clear==False:
            clear=True
            for b in range(len(buffer)):      
                try:
                    #if the next sequence in the buffer has a higher number of matches, remove current sequence
                    if int(sites[buffer[b]][0])<int(sites[buffer[b+1]][0]):
                        clear=False
                        buffer.pop(b)
                        break
                    #if the previous sequence in the buffer has a higher number of matches, remove current sequence
                    elif int(sites[buffer[b]][0])<int(sites[buffer[b-1]][0]):
                        clear=False
                        buffer.pop(b)
                        break
                except:
                    pass
            #select the middle one of the remainder
        sel.append(buffer[int(len(buffer)/2)])
    return sel

In [16]:
#Run first!
def reverse(Sequence):
    out=""
    for Base in Sequence:
        if Base == 'a':
            out='t'+ out
        elif Base == 't':
            out='a'+ out
        elif Base == 'c':
            out='g'+ out    
        elif Base == 'g':
            out='c'+ out
        elif Base == 's':
            out='w'+ out
        elif Base == 'w':
            out='s'+ out
        elif Base == 'k':
            out='m'+ out
        elif Base == 'm':
            out='k'+ out
        elif Base == 'r':
            out='y'+ out 
        elif Base == 'y':
            out='r'+ out
        elif Base == 'b':
            out='v'+ out
        elif Base == 'd':
            out='h'+ out
        elif Base == 'h':
            out='d'+ out
        elif Base == 'v':
            out='b'+ out
    return out


# Actual Code

In [22]:
#Site Locator
#Fasta list comparator that grabs raw data about each PAM site
infilename="consensus_23s.fasta"
infile=open(infilename,"r")
#Open and prep ALL-SITE outfile
outfilename= "sites_"+infilename.rstrip('.fasta')+"_ambiguity.txt"
outfile=open(outfilename,"w")
outfile.write("ID\tCount\tMatches\tStarts\tEnds\tF/R\trPAM\tSeq\tfPAM\n")
#threshold for ambiguity
athresh=3
#desired distance between sites
dist=50
#define variable for holding PAM site candidates
lets=0
#start line counter
lc=0
#start dictionary
#Dictionary structure
#Sequence:[Count,Matches,Starts,Ends,F/R,ID]
sites={}
#start site ID counters
rid=1
fid=1
#start loop to parse infile
for line in infile:
    #grab the sequence name from the FASTA format
    if lc%2 == 0:
        seqname= line.strip('>').rstrip('\n')
        print (seqname)
    #Analyze the sequence
    else:   
        sequence=line.rstrip('\n')
        #Start a loop that goes through the sequence one base at a time
        for win in range(len(sequence)):
            #Create a 23 BP site candidate
            lets=sequence[win:win+23]
            #make sure it is 23 BP (end of sequence detector)
            if(len(lets)==23):
                #if the first 2 bases are cc then it has a reverse PAM site "ccn"
                if lets[0:2] == 'cc':
                    matched='False'
                    #check if already recorded
                    for s in sites:
                        fval=fuzz(lets,s,athresh)
                        if fval != 'False':
                            #update information in the sites dictionary with information of new detection
                            num=sites[s][0]+1
                            match=sites[s][1]+', '+seqname
                            starts=str(sites[s][2])+', '+str(win)
                            ends=str(sites[s][3])+', '+str(win+23)
                            fr=sites[s][4]
                            sid=sites[s][5]
                            sites.pop(s)
                            sites[fval]=[num,match,starts,ends,fr,sid]
                            matched='True'
                            print('Matchedr',fval)
                            break
                    #add new PAM site to sites dictionary and provides a unique ID
                    if matched == 'False':
                        sites[lets]=[1,seqname,win,win+23,'R',('r'+str(rid))]
                        rid+=1
                        print('Newr')
                #same as above, but for the forwards PAM site "ngg"
                elif lets[21:23] == 'gg':
                    matched='False'
                    for s in sites:
                        fval=fuzz(lets,s,athresh)
                        if fval != 'False':
                            num=sites[s][0]+1
                            match=sites[s][1]+', '+seqname
                            starts=str(sites[s][2])+', '+str(win)
                            ends=str(sites[s][3])+', '+str(win+23)
                            fr=sites[s][4]
                            sid=sites[s][5]
                            sites.pop(s)
                            sites[fval]=[num,match,starts,ends,fr,sid]
                            matched='True'
                            print('Matchedf',fval)
                            break
                    if matched == 'False':
                        sites[lets]=[1,seqname,win,win+23,'F',('f'+str(fid))]
                        fid+=1
                        print('Newf')              
    lc+=1
infile.close()

Bif1_23s_con
Newf
Newf
Newf
Newf
Newr
Newr
Newr
Newf
Newf
Newf
Newr
Newf
Newf
Newf
Newr
Newr
Newr
Newr
Newf
Newf
Newf
Newf
Newr
Newf
Newr
Newr
Newr
Newf
Newr
Newr
Newr
Newr
Newr
Newf
Newr
Newr
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newr
Newr
Newr
Newr
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newr
Newf
Newr
Newr
Newr
Newr
Newr
Newf
Newf
Newf
Newr
Newr
Newf
Newf
Newf
Newf
Newr
Newr
Newr
Newr
Newf
Newf
Newr
Newf
Newf
Newf
Newr
Newf
Newf
Newr
Newf
Newf
Newf
Newf
Newf
Newf
Newr
Newr
Newf
Newf
Newr
Newf
Newf
Newf
Newr
Newr
Newf
Newf
Newr
Newr
Newf
Newf
Newr
Newf
Newf
Newr
Newr
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newr
Newf
Newr
Newf
Newr
Newr
Newr
Newf
Newf
Newf
Newr
Newf
Newf
Newf
Newf
Newf
Newr
Newr
Newr
Newr
Newr
Newf
Newf
Newr
Newf
Newf
Newf
Newf
Newf
Newr
Newf
Newr
Newr
Newr
Newr
Newf
Newf
Newf
Newr
Newr
Newf
Newf
Newf
Newf
Newf
Newf
Newr
Newr
Newf
Newf
Newf
Newf
Newf
Newr
Newf
Newr
Newr
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newf
Newr
Newf
Newf
Newr
Newf
Newf
Newf
Newf
Newr
Ne

Matchedr ccaaagagtaacggaggagyacg
Matchedf agagtaacggaggagyacgaagg
Newf
Newf
Newf
Newr
Newf
Newf
Newr
Newr
Newf
Matchedf gaaagyaggtcwtagtgatccgg
Matchedf agyaggtcwtagtgatccggtgg
Matchedf agtgatccggtggttctgwatgg
Matchedf atccggtggttctgwatggaagg
Matchedf tccggtggttctgwatggaaggg
Matchedr ccggtggttctgwatggaagggc
Matchedf atggaagggccatcgctcaacgg
Matchedf gccatcgctcaacggataaaagg
Matchedr ccatcgctcaacggataaaaggt
Newf
Matchedf tcaacggataaaaggtactcsgg
Matchedf caacggataaaaggtactcsggg
Newf
Matchedr ccgcccaagagttcatatcgacg
Matchedf cgcccaagagttcatatcgacgg
Matchedr cccaagagttcatatcgacggcg
Matchedr ccaagagttcatatcgacggcgg
Matchedf ttcatatcgacggcggtgtttgg
Matchedf ggtgtttggcacctcgatgtcgg
Matchedr cctcgatgtcggctcatcacatc
Matchedf gatgtcggctcatcacatcctgg
Matchedf atgtcggctcatcacatcctggg
Matchedf tgtcggctcatcacatcctgggg
Matchedf tcacatcctggggctgaagymgg
Matchedr cctggggctgaagymggtcccaa
Matchedf tggggctgaagymggtcccaagg
Matchedf ggggctgaagymggtcccaaggg
Matchedf tgaagymggtcccaagggtatgg
Newr
Matchedr cccaagg

Matchedr ccagctggctgcaactgtttawt
Matchedf gcactstgcaaacacgaaagtgg
Matchedf aacacgaaagtggacgtatasgg
Matchedf tatasggtgtgacgcctgcccgg
Matchedf gtgtgacgcctgcccggtgccgg
Matchedf gacgcctgcccggtgccggaagg
Matchedr cctgcccggtgccggaaggttaa
Matchedr cccggtgccggaaggttaattga
Matchedr ccggtgccggaaggttaattgat
Matchedf ggtgccggaaggttaattgatgg
Matchedf gtgccggaaggttaattgatggg
Matchedf tgccggaaggttaattgatgggg
Matchedr ccggaaggttaattgatggggty
Matchedf aattgatggggtyakccgyaagg
Matchedr ccgyaaggmgaagctcttgatcg
Matchedf gaagctcttgatcgaagccccgg
Matchedf ttgatcgaagccccggtaaacgg
Matchedf atcgaagccccggtaaacggcgg
Matchedr ccccggtaaacggcggccgtaac
Matchedr cccggtaaacggcggccgtaact
Matchedr ccggtaaacggcggccgtaacta
Matchedf aacggcggccgtaactataacgg
Matchedr ccgtaactataacggtcctaagg
Matchedr cctaaggtagcgaaattccttgt
Matchedf aaggtagcgaaattccttgtcgg
Matchedf aggtagcgaaattccttgtcggg
Matchedr ccttgtcgggtaagttccgacct
Matchedf taagttccgacctgcacgaatgg
Matchedr ccgacctgcacgaatggcgtaay
Matchedr cctgcacgaatggcgtaaygatg
Matchedf c

Newf
Matchedf sgtcgagcaggkacgaaagtmgg
Newf
Newf
Newf
Newf
Newr
Newf
Matchedf gcyrtcgctcaacggataaaagg
Matchedf tcaacggataaaaggtacyysgg
Matchedf caacggataaaaggtacyysggg
Matchedf aacggataaaaggtacyycgggg
Matchedf aaaggtacyycggggataacagg
Newr
Matchedf ygcccaagagtycatatcgacgg
Matchedr cccaagagtycatatcgacggcr
Matchedr ccaagagtycatatcgacggcrk
Newf
Newf
Newr
Matchedf rkkgtttggcacctcgatgtcgg
Matchedr cctcgatgtcggctcrtcrcatc
Matchedf gatgtcggctcrtcrcatcctgg
Matchedf atgtcggctcrtcrcatcctggg
Matchedf tgtcggctcrtcrcatcctgggg
Newf
Matchedf tcrcatcctggggctgragymgg
Matchedr cctggggctgragymggtcccaa
Matchedf tggggctgragymggtcccaagg
Matchedf ggggctgragymggtcccaaggg
Newf
Newf
Matchedr cccaagggtwkggctgttcgccm
Newr
Matchedf kggctgttcgccmwttaaagygg
Matchedr ccmwttaaagyggtacgcgagct
Newr
Matchedf mwttaaagyggtacgcgagctgg
Matchedf wttaaagyggtacgcgagctggg
Matchedf tagaacgtcgtgagacagttcgg
Newf
Newr
Newr
Newr
Newf
Newf
Matchedf tgmycctagtacgagaggaccgg
Newf
Newr
Matchedr cctagtacgagaggaccggrktg
Matchedf ctagtacgagagg

Matchedr cccattctggtgcggttagcgta
Matchedr ccattctggtgcggttagcgtat
Matchedf ggtgcggttagcgtattaacagg
Matchedf cgtattaacaggtgtgacgcagg
Matchedf ttaacaggtgtgacgcaggaagg
Matchedf gcaggaaggtagtccaagccagg
Matchedf aggtagtccaagccaggcgatgg
Matchedr ccaagccaggcgatggttgtcct
Matchedf aagccaggcgatggttgtcctgg
Matchedr ccaggcgatggttgtcctggtgc
Matchedf ttgtcctggtgcaagtgcgtagg
Matchedr cctggtgcaagtgcgtaggccga
Matchedf gtgcaagtgcgtaggccgaacgg
Matchedf aagtgcgtaggccgaacggtagg
Matchedr ccgaacggtaggcaaatccgccg
Matchedf ggcaaatccgccgttcacatagg
Matchedr ccgccgttcacataggctgagac
Matchedr ccgttcacataggctgagacacg
Matchedf acataggctgagacacgatgcgg
Matchedf gacacgatgcggataaaaagtgg
Matchedr cctatgctgccaagaaaagcatc
Matchedr ccaagaaaagcatcgacgcgagg
Matchedr ccgcccgtacccccaaaccgact
Matchedr cccgtacccccaaaccgactcag
Matchedr ccgtacccccaaaccgactcagg
Matchedf tacccccaaaccgactcaggtgg
Matchedr cccccaaaccgactcaggtggtc
Matchedr ccccaaaccgacwcaggtggtya
Matchedr cccaaaccgacwcaggtggtyag
Matchedr ccaaaccgacwcaggtggtyagg
Matchedr c

Matchedf accgtktgcttacaawccgtygg
Matchedr ccgtktgcttacaawccgtygga
Newr
Newf
Newf
Newr
Matchedr cctttgaagaatgagcctgcgag
Newr
Newf
Matchedf ygtsgcgaggttaacccgtgtgg
Matchedf gtsgcgaggttaacccgtgtggg
Matchedf tsgcgaggttaacccgtgtgggg
Matchedr cccgtgtggggtagccgtagcga
Matchedr ccgtgtggggtagccgtagcgaa
Matchedr ccgtagcgaaagcgagtctgaay
Matchedf tagcgaaagcgagtctkaayagg
Matchedf agcgaaagcgagtctkaayaggg
Newf
Newf
Matchedr cccgaagcgragtgatctaycca
Matchedr ccgaagcgragtgatctayccat
Matchedf gaagcgragtgatctayccatgg
Newf
Newf
Newr
Matchedr ccatggccrggttgaagcgacgg
Matchedr ccrggttgaagcgacggtaagac
Matchedf gaagcgacggtaagacgtcgtgg
Matchedf gcgacggtaagacgtcgtggagg
Matchedr ccgaacccacttmrgttgaaaah
Newf
Newr
Newr
Newf
Matchedf aaaaykgaggggatgagctgtgg
Newf
Matchedf kgaggggatgagctgtggrtagg
Matchedf gaggggatgagctgtggrtaggg
Matchedf aggggatgagctgtggrtagggg
Newf
Matchedr ccaatcaaactycgtgatagctg
Matchedf caatcaaactycgtgatagctgg
Newr
Matchedf gttctcyccgaaatgcatttagg
Newr
Newr
Matchedr ccgaaatgcatttaggtgcagcg
Matchedf 

Newf
Newr
Newr
Newr
Matchedf ktatcttgtactgaatacatagg
Matchedf actgaatacataggtgcaagagg
Matchedf ataggtgcaagaggcgaaccagg
Matchedf taggtgcaagaggcgaaccaggg
Matchedf aggtgcaagaggcgaaccagggg
Matchedr ccrggggaactgaaacatctmag
Matchedf tgaaacatctmagtacccnsagg
Matchedr cccygaggaaaagaaatcaaccg
Matchedr ccygaggaaaagaaatcaaccga
Matchedf aaccgagattcccyyagtagygg
Matchedr ccgagattcccyyagtagyggcg
Matchedr ccsttagtagtggcgagcgaams
Matchedr ccyyagtagyggcgagcgaacgg
Matchedf ccyagtagyggcgagcgaacggg
Matchedf ttagtagtggcgagcgaamssgg
Newf
Matchedr cccttaagtkgstttgagattag
Matchedr ccttaagtkgstttgagattagy
Matchedf ttaagtkgstttgagattagygg
Matchedf ttgagattagyggaacgctctgg
Matchedf gyggaacgstctggaaagtscgg
Matchedf tctggaaagtgcggccatagtgg
Matchedf ctggaaagtgcggccatagtggg
Matchedr ccatagtgggtgatagcccygta
Matchedr cccygtacrcgaaaatctcttak
Matchedr ccygtacrcgaaaatctcttaky
Matchedf tcttakymatgaaatcgagtagg
Matchedf akymatgaaatcgagtaggacgg
Newf
Newf
Matchedf cgagaaacyttgtctgaayatgg
Matchedf gagaaacyttgtctgaayatggg
Matchedf

Matchedf ymtggtgggtagtttractgggg
Matchedf grtgggtagtttractggggcgg
Matchedf ggtctcctccyaaagmgtaacgg
Matchedf ctcctccyaaagmgtaacggagg
Matchedr cctcctaaaragtaacggaggmg
Matchedr ccyaaagmgtaacggaggagyac
Matchedf agmgtaacggaggagyacgaagg
Matchedf gtacgaaggtgcgctcagaccgg
Matchedf gaaggtgcgctcagaccggtcgg
Matchedf cgctcagaccggtcggaaatcgg
Matchedr ccggtcggaaatcggtcgtagag
Matchedf aaatcggtcgtagagtataaagg
Matchedf gcgagacmsacacgtcgagcagg
Matchedf sgtcgagcaggkacgaaagtmgg
Matchedf gaaagyaggtcwtagtgatccgg
Matchedf agyaggtcwtagtgatccggtgg
Matchedf agtgatccggtggttctgwatgg
Matchedf atccggtggttctgwatggaagg
Matchedf tccggtggttctgwatggaaggg
Matchedr ccggtggttctgwatggaagggc
Matchedf rtggaakggccrtcgctcaacgg
Matchedf gcyrtcgctcaacggataaaagg
Matchedr ccrtcgctcaacggataaaaggt
Matchedf tcaacggataaaaggtacyysgg
Matchedf caacggataaaaggtacyysggg
Matchedf aacggataaaaggtacyycgggg
Matchedf aaaggtacyycggggataacagg
Matchedr ccggggataacaggctgataccg
Matchedr ccgcccaagagttcatatcgacg
Matchedf ygcccaagagtycatatcgacgg
Matchedr c

Matchedf wyccggtggttctgwatggaagg
Matchedr yccggtggttctgwatggaaggg
Matchedr ccggtggttctgwatggaagggc
Matchedf rtggaakggccrtcgctcaacgg
Matchedf gcyrtcgctcaacggataaaagg
Matchedr ccrtcgctcaacggataaaaggt
Matchedf tcaacggataaaaggtacyybgg
Matchedf caacggataaaaggtacyybggg
Matchedf aacggataaaaggtacyyygggg
Matchedf aaaggtacyyyggggataacagg
Newr
Newf
Newr
Matchedr ccmamgagtycatatcgacggcr
Newf
Matchedf ttcatatcgacggcggtgtttgg
Matchedf rkkgtttggcacctcgatgtcgg
Matchedr cctcgatgtcggctcrtcdcatc
Matchedf gatgtcggctcrtcdcatcctgg
Matchedf atgtcggctcrtcdcatccysgg
Matchedf tgtcggctcrtcdcatcctgggg
Matchedf tcdcatcctggggctgdagymgg
Matchedr ccyggggctgdagymggtcccaa
Matchedf tggggctgdagymggtcccaagg
Matchedf ggggctgdagymggtcccaaggg
Newf
Newf
Newr
Newr
Newr
Matchedf kggctgttcgccmwttaaaghgg
Matchedr ccmwttaaaghggtacgygagct
Newf
Matchedf wttaaaghggtacgygagctggg
Matchedf taraacgtcgtgagacagttygg
Matchedf gttyggtccctatctgccskkgg
Newf
Newr
Newr
Newr
Newr
Matchedf cgtgggcgytggakawttgargg
Newf
Newf
Newf
Matchedf ggggytgctc

Matchedf gcgatgggggggacggatcgcgg
Matchedf tgggggggacggatcgcggaagg
Matchedf acggatcgcggaaggttgtccgg
Matchedf cggatcgcggaaggttgtccggg
Matchedf gcggaaggttgtccgggtgttgg
Matchedf tgtccgggtgttggaagtcccgg
Matchedr ccgggtgttggaagtcccggtcg
Matchedf ggaagtcccggtcgctgcattgg
Matchedr cccggtcgctgcattggcagaag
Matchedr ccggtcgctgcattggcagaagg
Matchedf tgcattggcagaaggcgcttagg
Matchedf agaaggcgcttaggcaaatccgg
Matchedf gaaggcgcttaggcaaatccggg
Matchedf cgcttaggcaaatccgggcgcgg
Matchedf aaatccgggcgcggaattcaagg
Matchedf aatccgggcgcggaattcaaggg
Matchedr ccgggcgcggaattcaagggtgt
Matchedf gggcgcggaattcaagggtgtgg
Matchedf gggtgtggcgcgagctccttagg
Matchedf ggtgtggcgcgagctccttaggg
Newf
Newr
Newf
Matchedr ccaagaaaagcyyctaagcttca
Matchedr cctctaagcttcagtctaacgat
Matchedf mycgtaccscaaaccgacacagg
Matchedr ccgtaccscaaaccgacacaggt
Matchedf gtaccscaaaccgacacaggtgg
Matchedf taccgcaaaccgacacaggtggg
Matchedr ccscaaaccgacacaggtggkys
Matchedr ccgacacaggtgggccgagatga
Matchedf gggccgagatgagtattctaagg
Matchedr ccgagatgagtattctaag

In [23]:
#Conserved Site finder pt. 1
#Create a temporary list [sequence, average start, sequence 2, average start 2...]
temp=[]
#sorting algorithm variable
ordered=False
#loop through all sites
for s in sites:
    #select sites that have a count that passes a certain number
    if sites[s][0] >= 4: #lc/4
        #take an average of the start position and add them to the temp list
        ave=sites[s][2].split(',')
        Sum=0
        count=0
        for i in range(len(ave)):
            Sum+=int(ave[i])
            count+=1
        average=Sum/count
        temp.append(average)
        temp.append(s)
#sort the temp list using insertion sort        
while ordered == False:
    ordered=True
    for t in range(len(temp)):
        if t > 1:
            if t % 2 == 0:
                if temp[t-2]>temp[t]:
                    ordered=False
                    for u in range(len(temp)):
                        if u % 2 == 0:
                            if temp[t]<temp[u]:
                                temp.insert(u,temp[t+1])
                                temp.insert(u,temp[t+1])
                                temp.pop(t+2)
                                temp.pop(t+2)
                                break
        if ordered == False:
            break
print (len(temp)/2)

259.0


In [24]:
#Conserved Site finder pt. 2
#start a buffer list
buffer=[]
#final sites dictionary
final={}
#clear newin
newin=0
#loop thrpough temp list
for m in range(len(temp)):
    #the last one breaks it
    if m< len(temp)-2:
        #we only need to look at the number so select every other one
        if m %2 ==0:
            #if the buffer is too long select a sequence
            
            #if the next start value is within 23 bp add it to the buffer
            if temp[m]<temp[m+2]<(temp[m]+23):
                buffer.append(temp[m+1])
                print (temp[m], sites[temp[m+1]][0],'if')
                if newin==0:
                    newin=float(temp[m])
            # if there is nothing within 23 bp and the buffer is empty, add sequence to the final dictionary
            elif len(buffer)==0:
                final[temp[m+1]]=[sites[temp[m+1]][0],sites[temp[m+1]][1],sites[temp[m+1]][2],sites[temp[m+1]][3],sites[temp[m+1]][5],sites[temp[m+1]][4]]
                print (temp[m],'elif')
                newin=0
            #if there is something in the buffer
            else:
                print (temp[m],'else')
                sel=select(buffer,newin,temp[m])
                #add selection to final dictionary
                for s in sel:
                    final[s]=[sites[s][0],sites[s][1],sites[s][2],sites[s][3],sites[s][5],sites[s][4]]
                print (sel)
                #clear buffer
                buffer=[]
                newin=0
            print(float(temp[m])-newin>=dist)

sel=select(buffer,newin,temp[m-1])
#add selection to final dictionary
for s in sel:
    final[s]=[sites[s][0],sites[s][1],sites[s][2],sites[s][3],sites[s][5],sites[s][4]]
print (sel)
#clear buffer
buffer=[]
print (final)
print (len(final))

21.0 4 if
False
31.0 4 if
False
37.5 4 if
False
38.0 4 if
False
42.5 4 if
False
62.5 4 if
False
63.5 else
['ccyhggcagtcagaggcgatgaa']
True
104.5 elif
True
190.0 4 if
False
200.14285714285714 7 if
False
214.6 5 if
False
215.0 4 if
False
230.0 5 if
False
232.0 5 if
False
242.83333333333334 6 if
True
243.5 4 if
True
244.25 4 if
True
246.5 else
['ccyyagtagyggcgagcgaacgg']
True
304.5 4 if
False
317.5 else
['ctggaaagtgcggccatagyrgg']
True
392.0 4 if
False
393.0 4 if
False
394.0 4 if
False
395.0 4 if
False
396.0 4 if
False
400.0 4 if
False
406.25 4 if
False
417.1666666666667 6 if
False
421.1666666666667 6 if
False
423.25 else
['cctccaaggctaaatactmstga']
True
474.3333333333333 6 if
False
475.8 5 if
False
476.1111111111111 9 if
False
476.8 5 if
False
478.1111111111111 9 if
False
478.1666666666667 6 if
False
479.1111111111111 9 if
False
484.1111111111111 9 if
False
485.875 8 if
False
488.25 4 if
False
491.875 8 if
False
504.8 5 if
False
512.25 4 if
False
514.5 4 if
False
517.25 4 if
False
518.25

### Don't use this

In [ ]:
#2nd pass
#Output Goal
#seqname, ID, start, end, F/R, rpam, seq, fpam
#final fmt
#[num,seqname,starts,end,ID,F/R]
#sites fmt
#Sequence:[Count,Matches,Starts,Ends,F/R,ID]
#create new list structured [sequence,count,start]
finalists=[]
#new line counter
lc=0
#"previous sequence" values to prime the functions
lastend=999999999999
lastseq=''
#reopen infile
infile=open(infilename,"r")
#debugin pseudocode
debug=0
#start looping through the file, same as the beginning
for line in infile:
    if lc%2 == 0:
        seqname= line.strip('>').rstrip('\n')
        print (seqname)
    else:   
        #print ('else')
        sequence=line.rstrip('\n')
        found='False'
        #loop through each sequence
        for win in range(len(sequence)):
            debug+=1
            lets=sequence[win:win+23]
            if(len(lets)==23):
                #check if section of sequence is in final
                if lets in final:
                    print('returner')
                    
#removed fragment of code that would need ambiguous bases to be used
    #                if lastend<win:
   #                 while
  #                      print('overruled')
 #                       final.pop(lastseq)                                      
#                       newloc=int(len(finalists)/2)
 #                      newseq=finalists[newloc]
  #                     if newseq not in final:
   #                        final[newseq]=[1,seqname,finalisits[newseq][1],finalisits[newseq][2],sites[newseq][5],sites[newseq][4]]
    #                   else:
     #                      num=final[newseq][0]+1
      #                     match=final[newseq][1]+', '+seqname
       #                    starts=str(final[newseq][2])+', '+str(finalisits[newseq][1])
        #                   ends=str(final[newseq][3])+', '+str(finalisits[newseq][2])
         #                  del final[newseq][0:4]
          #                 final[newseq].insert(0,ends)
           #                final[newseq].insert(0,starts)
            #               final[newseq].insert(0,match)
             #              final[newseq].insert(0,num)
                 
                    
                    #clear finalists because it is no longer relevant
                    finalists=[]
                    #if this sequence is new, update the information in final
                    if seqname not in final[lets][1]:
                        num=final[lets][0]+1
                        match=final[lets][1]+', '+seqname
                        starts=str(final[lets][2])+', '+str(win)
                        ends=str(final[lets][3])+', '+str(win+23)
                        del final[lets][0:4]
                        final[lets].insert(0,ends)
                        final[lets].insert(0,starts)
                        final[lets].insert(0,match)
                        final[lets].insert(0,num)
                    #set flag to 'True' so the next part will be skipped 
                    found='True'
                    #update last info
                    lastend= win+23
                    lastseq= lets
                #if it is a PAM site, add it to the finalists list
                elif lets in sites:
                    finalists.append(lets)
                    finalists.append(sites[lets][0])
                    finalists.append(win)
                #every distance partition check finalists
                if (win % dist == 0) and (win != 0):
                    #If no previous match was found, select a new match
                    if found=='False':    
                        print(win)
                        #reset values
                        best=0
                        bests=''
                        bestw=0
                        #cycle through finalists
                        for f in range(len(finalists)):
                            #print(f)
                            #if there is only one in finalists, select it
                            if len(finalists)==3:
                                if f % 3 == 1:
                                    best = finalists[f]
                                    bests = finalists[f-1]
                                    bestw = finalists[f+1]
                            #else, sift through finalists three at a time
                            elif f<=(len(finalists)-3):
                                if f % 3 == 1:
                                    #print('count')
                                    #print(finalists[f])
                                    #print(finalists[f+3])
                                    #compare value to the next one and the best
                                    if finalists[f]>finalists[f+3]:
                                        if finalists[f]>best:
                                            #if it beats both, update best values
                                            best = finalists[f]
                                            bests = finalists[f-1]
                                            bestw = finalists[f+1]
                                            print (best, bests, bestw)
                                    else:
                                        #this is here in case the final value is the best because it would be skipped in the loop
                                        if finalists[f+3]>best:
                                            best = finalists[f+3]
                                            bests = finalists[f+2]
                                            bestw = finalists[f+4]
                                            print (best, bests, bestw)
                        #one last check then it is added to final
                        if bests not in final:
                            print('new')
                            final[bests]=[1,(seqname),bestw,(int(bestw)+23),sites[bests][5],sites[bests][4]]
                    finalists=[]
                    found='False'
    lc+=1
print (final)
infile.close()

# Outputs

In [97]:
#All sites Output
for s in sites:
    #"Count\tMatches\tStarts\tEnds\tF/R\trPAM\tSeq\tfPAM\n"
    if sites[s][4] == 'F':
        Outline='%s\t%i\t%s\t%s\t%s\t%s\t\t%s\t%s\n'% (sites[s][5],sites[s][0],sites[s][1],sites[s][2],sites[s][3],sites[s][4],s[0:20],s[20:23])
    elif sites[s][4] == 'R':
        Outline='%s\t%i\t%s\t%s\t%s\t%s\t%s\t%s\t\n'% (sites[s][5],sites[s][0],sites[s][1],sites[s][2],sites[s][3],sites[s][4],s[0:3],s[3:23])
    outfile.write(Outline)
outfile.close()

ValueError: I/O operation on closed file.

In [94]:
#Post-selection Verbose Outputter
#Output Goal
#seqname, ID, start, end, F/R, rpam, seq, fpam
#final fmt
#[num,seqname,starts,end,ID,F/R]
#sites fmt
#Sequence:[Count,Matches,Starts,Ends,F/R,ID]
outfilename2="sites_"+infilename.rstrip('.fasta')+"_parsed.txt"
outfile2=open(outfilename2,'w')
outfile2.write("Sequence Name\tNumber\tSite ID\tStart\tEnd\tF/R\tReverse PAM\tSequence\tForward PAM\n")
#sift through final
for f in final:
    #if there is more than one match for the sequence
    if final[f][0]>1:
        #loop through those values and output them individually
        for g in range(final[f][0]):
            seqnames=final[f][1].split(', ')
            seqname=seqnames[g]
            ID=final[f][4]
            starts=final[f][2].split(', ')
            start=starts[g]
            ends=final[f][3].split(', ')
            end=ends[g]
            FR=final[f][5]
            if FR == 'F':
                rpam=''
                seq=f[0:20]
                fpam=f[20:23]
            elif FR== 'R':
                rpam=f[0:3]
                seq=f[3:23]
                fpam=''
            Outline='%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (seqname,final[f][0], ID, start, end, FR, rpam, seq, fpam)
            print (Outline)
            outfile2.write(Outline)
    #otherwise just add it to the file
    else:
        seqname=seqnames[1]
        ID=final[f][4]
        start=final[f][2]
        end=final[f][3]
        FR=final[f][5]
        if FR == 'F':
            rpam=''
            seq=f[0:20]
            fpam=f[20:23]
        elif FR== 'R':
            rpam=f[0:3]
            seq=f[3:23]
            fpam=''
        Outline='%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % (seqname, final[f][0], ID, start, end, FR, rpam, seq, fpam)
        print(Outline)
        outfile2.write(Outline)
outfile2.close()


chlorp7_16s	7	f147	12	35	F		yggctcagrwtgaacgctgg	cgg

Diaz23_16s	7	f147	19	42	F		yggctcagrwtgaacgctgg	cgg

Imid1W_16s	7	f147	12	35	F		yggctcagrwtgaacgctgg	cgg

Imid2_16s	7	f147	1	24	F		yggctcagrwtgaacgctgg	cgg

Mal2_16s	7	f147	9	32	F		yggctcagrwtgaacgctgg	cgg

Perm7_16s	7	f147	12	35	F		yggctcagrwtgaacgctgg	cgg

E. coli	7	f147	19	42	F		yggctcagrwtgaacgctgg	cgg

Bif1_16s	7	r1	26	49	R	cct	wacacatgcaagtcgarcgg	

chlorp7_16s	7	r1	39	62	R	cct	wacacatgcaagtcgarcgg	

Diaz23_16s	7	r1	46	69	R	cct	wacacatgcaagtcgarcgg	

Imid1W_16s	7	r1	40	63	R	cct	wacacatgcaagtcgarcgg	

Mal2_16s	7	r1	36	59	R	cct	wacacatgcaagtcgarcgg	

Perm7_16s	7	r1	39	62	R	cct	wacacatgcaagtcgarcgg	

E. coli	7	r1	46	69	R	cct	wacacatgcaagtcgarcgg	

Bif1_16s	5	f9	83	106	F		cggacgggtgagtaatgyct	rgg

chlorp7_16s	5	f9	92	115	F		cggacgggtgagtaatgyct	rgg

Diaz23_16s	5	f9	105	128	F		cggacgggtgagtaatgyct	rgg

Mal2_16s	5	f9	91	114	F		cggacgggtgagtaatgyct	rgg

E. coli	5	f9	104	127	F		cggacgggtgagtaatgyct	rgg

Bif1_16s	6	f30	257	280	F		cgatc

In [26]:
#Post-Selection Oligo Pure Output
outfilename3="sites_"+infilename.rstrip('.fasta')+"_pure.txt"
outfile3=open(outfilename3,'w')
for f in final:
    if final[f][5] == 'F':
        seq=f[0:20]
        print ('F')
        print (f)
        print (seq)
    elif final[f][5] == 'R':
        seq=reverse(f[3:23])
        print ('R')
        print (f)
        print ('   '+seq)
    outfile3.write("taatacgactcactatag"+seq+"gtttaagagctatgctggaaac\n")
                #   TAATACGACTCACTATAG       GTTTAAGAGCTATGCTGGAAAC
outfile3.close()


R
ccyhggcagtcagaggcgatgaa
   ttcatcgcctctgactgccd
R
ccsdrgrtgtccgaatggggaaa
   tttccccattcggacaycyh
R
ccyyagtagyggcgagcgaacgg
   ccgttcgctcgccrctactr
F
ctggaaagtgcggccatagyrgg
ctggaaagtgcggccatagy
R
cctccaaggctaaatactmstga
   tcawkagtatttagccttgg
F
atagygrachagtaccgtgaggg
atagygrachagtaccgtga
F
gaaggttrggtaacactractgg
gaaggttrggtaacactrac
F
aaaarttagcggatgacytgtgg
aaaarttagcggatgacytg
R
ccycgaaagctatttaggtagcg
   cgctacctaaatagctttcg
F
gtagagcactgtttcggctaggg
gtagagcactgtttcggcta
R
ccgatgcaaactmcgaataccka
   tmggtattcgkagtttgcat
F
tgctaacgtycgtcgtgaaragg
tgctaacgtycgtcgtgaar
R
cccagaccrycarctaaggtccc
   gggaccttagytgryggtct
F
kcctgcgcggaagatgtaacggg
kcctgcgcggaagatgtaac
F
agcgttctgtaagcckkygaagg
agcgttctgtaagcckkyga
F
tgacatragtarcgataawgsgg
tgacatragtarcgataawg
R
ccaaggkttcctgyscaacgtta
   taacgttgwrcaggaamcct
R
ccgaaagaccaaggkttccwgsg
   cwcsggaamccttggtcttt
F
argcgtagtcgatggrmaacrgg
argcgtagtcgatggrmaac
F
gtaccscaaaccgacacaggtgg
gtaccscaaaccgacacagg
F
agaactcgggwgaaggaactmgg
agaactc

# Various Tools

In [16]:
#single sequence search
infilename="Ecoli16s.txt"
infile=open(infilename,"r")
outfilename= infilename.rstrip('.txt')+".csv"
outfile=open(outfilename,"w")
lets=0
outfile.write("Start,End,F/R,rPAM,Seq,fPAM\n")
for line in infile:
    sequence=line.rstrip('\n')
    for win in range(len(sequence)):
        lets=sequence[win:win+23]
        if(len(lets)==23):
            if lets[0:2] == 'CC':
                Outline="%s,%s,Reverse,%s,%s,\n" % (win, int(win)+23,lets[0:3],lets[3:23])
                outfile.write(Outline)
            if lets[21:23] == 'GG':
                Outline="%s,%s,Forward,,%s,%s\n" % (win, int(win)+23,lets[0:20],lets[20:23])
                outfile.write(Outline)
infile.close()
outfile.close()

In [12]:
#reverse reverser, mark all R strands with <
infilename="consensus_23s.fasta"
infile=open(infilename,'r')
outfilename="consensus_23sa.fasta"
outfile=open(outfilename,'w')
for line in infile:
    if line[0] == '<':
        Outline=reverse(line)+'\n'
        print ("Reverse!")
    else:
        Outline=line
    outfile.write(Outline)
infile.close()
outfile.close()

Reverse!
Reverse!
Reverse!
Reverse!


In [83]:
#fastafier
infilename='DASH/Perm7_23s.txt'
infile=open(infilename,'r')
outfilename=infilename.rstrip('.txt')+'.fasta'
outfile=open(outfilename,'w')
for line in infile:
    fields=line.rstrip('\n').split('\t')
    outline= '>'+fields[0]+'\n'+fields[1]+'\n'
    outfile.write(outline)
infile.close()
outfile.close()